## Load latest headlines

In [1]:
# ========== Bootstrap: ensure required Python packages are present =========
import importlib, subprocess, sys
from typing import Optional

def _ensure(pkg_name: str, import_name: Optional[str] = None):
    try:
        importlib.import_module(import_name or pkg_name)
    except ModuleNotFoundError:
        subprocess.check_call([sys.executable, '-m', 'pip', 'install', pkg_name])
    finally:
        globals()[import_name or pkg_name] = importlib.import_module(import_name or pkg_name)

_ensure('pandas')
print('All dependencies ready.\n')


All dependencies ready.



In [2]:
from pathlib import Path
archive = Path('archive')
archive.mkdir(exist_ok=True)
import pandas as pd
latest = pd.read_csv('../headlines/latest.csv')
latest.head()

,pubdate,source,title,link
0,2026-01-12-08-57-07 +0000,bbc,Nigerian stars dominate All Africa Music Awards,https://www.bbc.com/news/articles/c5y2gkydpp5o...
1,2026-01-12-08-55-00 +0000,wsj,Increased Fed Interference Could Be Precious M...,https://www.wsj.com/finance/commodities-future...
2,2026-01-12-08-39-20 +0000,bbc,India and US 'actively engaged' in trade negot...,https://www.bbc.com/news/articles/c4gr6e6dn5lo...
3,2026-01-12-08-37-00 +0000,wapo,Trump says Tehran wants to talk as U.S. weighs...,https://www.washingtonpost.com/world/2026/01/1...
4,2026-01-12-08-18-00 +0000,wsj,China’s Daily Stock Trading Volume Hits Record...,https://www.wsj.com/finance/stocks/chinas-dail...


## Calculate word frequencies

In [3]:
import re
from collections import Counter
from datetime import datetime

with open('exclude.txt') as f:
    stop_words = set(w.strip() for w in f if w.strip())
words = re.findall(r'[A-Za-z]+', ' '.join(latest['title']).lower())
filtered = [w for w in words if w not in stop_words and len(w) > 1]
counts = Counter(filtered)
score_df = (
    pd.DataFrame(counts.items(), columns=['word','score'])
    .sort_values('score', ascending=False)
)
score_df[['score','word']].to_csv('scores.csv', index=False)
timestamp = datetime.utcnow().strftime('%Y-%m-%d-%H-00-00')
score_df[['score','word']].to_csv(archive / f'scores-{timestamp}.csv', index=False)
score_df.head()


/tmp/ipykernel_2381/2220774760.py:15: DeprecationWarning: datetime.datetime.utcnow() is deprecated and scheduled for removal in a future version. Use timezone-aware objects to represent datetimes in UTC: datetime.datetime.now(datetime.UTC).
  timestamp = datetime.utcnow().strftime('%Y-%m-%d-%H-00-00')


,word,score
21,trump,35
65,iran,18
344,ice,11
239,shooting,8
368,oil,7


## Rank headlines by score

In [4]:
word_scores = dict(score_df[['word','score']].values)
latest['score'] = latest['title'].apply(
    lambda t: sum(
        word_scores.get(w.lower(), 0)
        for w in re.findall(r'[A-Za-z]+', t)
        if len(w) > 1
    )
)
ranked = latest.sort_values('score', ascending=False)
ranked[['score','pubdate','source','title','link']].to_csv('rank.csv', index=False)
ranked[['score','pubdate','source','title','link']].to_csv(archive / f'rank-{timestamp}.csv', index=False)
ranked.head()


,pubdate,source,title,link,score
21,2026-01-12-05-36-07 +0000,bbc,Trump mulls 'very strong' military options as ...,https://www.bbc.com/news/articles/czdqy3rmp78o...,82
88,2026-01-11-22-39-27 +0000,nyt,Deposed Shah’s Son Hopes Trump Will Put Iran R...,https://www.nytimes.com/2026/01/11/world/europ...,75
9,2026-01-12-07-50-32 +0000,nypost,Trump says he will talk to Musk about restorin...,https://nypost.com/2026/01/12/us-news/trump-sa...,72
144,2026-01-11-16-29-40 +0000,cbc,Trump says no more Venezuelan oil or money to ...,https://www.cbc.ca/news/world/trump-says-no-mo...,70
101,2026-01-11-21-55-03 +0000,nyt,Trump Says Cuba Will No Longer Get ‘Oil or Mon...,https://www.nytimes.com/2026/01/11/us/politics...,69


## Select top headlines

In [5]:
with open('exclude_sources.txt') as f:
    excluded_sources = {
        line.strip()
        for line in f
        if line.strip() and not line.strip().startswith('#')
    }

top_count = 10
top_rows = []
working = word_scores.copy()
remaining = latest.copy()

while len(top_rows) < top_count and not remaining.empty:
    scored_remaining = remaining.assign(
        score=remaining['title'].apply(
            lambda t: sum(
                working.get(w.lower(), 0)
                for w in re.findall(r'[A-Za-z]+', t)
                if len(w) > 1
            )
        )
    )
    ranked_loop = scored_remaining.sort_values('score', ascending=False)

    if ranked_loop.empty:
        break

    top_story = ranked_loop.iloc[0]
    remaining = remaining.drop(top_story.name)

    if top_story['source'] in excluded_sources:
        continue

    words = set(re.findall(r'[A-Za-z]+', top_story['title'].lower()))
    for word in words:
        working.pop(word, None)

    top_rows.append(top_story[['score', 'pubdate', 'source', 'title', 'link']])

top_df = pd.DataFrame(top_rows)
top_df.to_csv('top.csv', index=False)
top_df.to_csv(archive / f'top-{timestamp}.csv', index=False)
top_df


,score,pubdate,source,title,link
21,82,2026-01-12-05-36-07 +0000,bbc,Trump mulls 'very strong' military options as ...,https://www.bbc.com/news/articles/czdqy3rmp78o...
18,61,2026-01-12-05-56-00 +0000,wsj,The criminal investigation into Federal Reserv...,https://www.wsj.com/economy/central-banking/tr...
89,51,2026-01-11-22-37-17 +0000,nypost,"NYC anti-ICE protest draws 2,000 in latest unr...",https://nypost.com/2026/01/11/us-news/nyc-anti...
144,35,2026-01-11-16-29-40 +0000,cbc,Trump says no more Venezuelan oil or money to ...,https://www.cbc.ca/news/world/trump-says-no-mo...
97,29,2026-01-11-22-09-05 +0000,nypost,Top Minnesota GOP rep. hopes Tim Walz ‘leaves ...,https://nypost.com/2026/01/11/us-news/top-minn...
163,28,2026-01-11-13-00-00 +0000,nypost,After Marine vet Trevor Reed was freed from a ...,https://nypost.com/2026/01/11/world-news/marin...
48,27,2026-01-12-02-33-58 +0000,nypost,Black New York homeowners blast Mamdani’s radi...,https://nypost.com/2026/01/11/us-news/black-ne...
27,26,2026-01-12-05-00-04 +0000,nypost,Arkansas mother killed alongside her 6-year-ol...,https://nypost.com/2026/01/12/us-news/arkansas...
133,24,2026-01-11-18-25-23 +0000,nypost,US invasion of Greenland would trigger ‘end of...,https://nypost.com/2026/01/11/us-news/us-invas...
65,24,2026-01-12-00-45-34 +0000,bbc,"Meta blocks 550,000 accounts under Australia's...",https://www.bbc.com/news/articles/cpqye2yygl4o...


## Create JSON version of top headlines

In [6]:
import pandas as pd
pd.read_csv('top.csv').to_json('top.json', orient='records', indent=2)
